entrainer modèle pour classifier :
    -classification personelle
- associer les phrases à des cas féminins masculins ou neutre (classes)
ensuite : effacer les noms et voir si ça marche tjr
    => test de prédiction

- prendre des sujets : ex : faire un régime
    voir les thématiques associées à des femmes
    quand je fait la détection, ne pas utiliser les noms propres différents (NOM1)

    ensuite, inverser nom féminin et masculin et tester la réussite du schéma winogrande
    1 nom féminin, masculin  ou les 2

environ 200-400 annotation
éventuellement utiliser des recherches de mots (pas trop généraux et plusieurs)

utiliser : sklearn => word embeding (BERT)

ensuite : validation croisée , couper le corpus en pluseurs morceaux et faire le test plusieurs fois, car on manque de données
=> k fold

faire des stats sur les sous ensemble d'annotation

prendre RoBerta finetuné sur winograde et lui proposer de items non biaisé à résoudre et comparer les scores avecdes items biaisés

pour bert: faire en sorte que ça donne bien les embedding

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving phrases_annotees.csv to phrases_annotees.csv


In [ ]:
# import numpy as np
# from sklearn.model_selection import KFold
# X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
# y = np.array([1, 2, 3, 4])
# kf = KFold(n_splits=2)
# kf.get_n_splits(X)
# print(kf)
# for i, (train_index, test_index) in enumerate(kf.split(X)):
#     print(f"Fold {i}:")
#     print(f"  Train: index={train_index}")
#     print(f"  Test:  index={test_index}")

sport
caring
wealth
advice

hesitation, question, advice
performance, sucess, competence
Failure
comparison of performance, road to sucess
relationship, helping

aussi : comptabiliser her/his pour les femmes/h

In [ ]:
# # Nom du fichier source et du fichier de sortie
# fichier_entree = "train.txt"
# fichier_sortie = "train_cut.txt"

# # Lecture du fichier et filtrage
# with open(fichier_entree, "r", encoding="utf-8") as f_in:
#     lignes = f_in.readlines()

# # Garder une ligne sur deux
# lignes_filtrees = lignes[::2]  # [::2] = toutes les 2 lignes en partant de 0

# # Écriture du résultat dans un nouveau fichier
# with open(fichier_sortie, "w", encoding="utf-8") as f_out:
#     f_out.writelines(lignes_filtrees)

# print(f"{len(lignes_filtrees)} lignes conservées dans '{fichier_sortie}'.")


In [ ]:
df

,qID,phrase,option1,option2,answer,label
0,3QHITW7OYO7Q6B6ISU2UMJB84ZLAQE-1,Ian volunteered to eat Dennis's menudo after a...,Ian,Dennis,1,0
1,3NRZ1LDP7W4677DB493437C7D3WZP2-1,I picked up a bag of peanuts and raisins for a...,raisins,peanuts,1,0
2,3S4TINXCC217YYX13TMMKNON1TXBOG-1,Eric was teaching all of the proper stretches ...,Eric,Neil,1,-1
3,3EHVO81VN5JHZUUFDLM724Q4E0O1HV-1,Bill poured the entire bowl of cheese snack in...,mouth,bowl,1,-1
4,3J94SKDEKINZBWQWSC5TE9ONT9BD5S-1,Nick had a bigger dog than Craig because _ fed...,Nick,Craig,1,-1
...,...,...,...,...,...,...
1730,3WGCNLZJKHMFH3OKBNFH3HZFCBQ1DY-1,"At the meat buffett, the man decided to eat th...",beans,noodles,1,-1
1731,3ZCC2DXSD720BVV7ZAX42RWHBK8YYT-1,Adam placed the fruit into the bag instead of ...,bag,box,1,0
1732,38EHZ67RIO6FPCK06ZZNTUQFM97GMQ-1,I took the food out of the oven and placed it ...,oven,bowl,1,-1
1733,38F60IALAIVMR3ZZ1UBWUGYR9PYT0Z-1,The scale got broken when the bag of sugar was...,scale,bag,1,-1


In [ ]:
import pandas as pd

# Charger le fichier CSV
df = pd.read_csv("phrases_annotees.csv")  # Remplace par le nom réel

# Filtrer les colonnes nécessaires
liste_2D = df[["phrase", "label"]].values.tolist()

# Afficher un exemple
for ligne in liste_2D[:5]:
    print(ligne)

liste_2D_bis = []
for ligne in liste_2D:
  if not ligne[1] == -1:
    liste_2D_bis.append(ligne)

liste_2D = liste_2D_bis

["Ian volunteered to eat Dennis's menudo after already having a bowl because _ enjoyed eating intestine.", 0]
['I picked up a bag of peanuts and raisins for a snack. I wanted a sweeter snack out so I ate the _ for now.', 0]
['Eric was teaching all of the proper stretches to Neil, because _ was using them to get paid.', -1]
['Bill poured the entire bowl of cheese snack into his mouth until the _ was full.', -1]
['Nick had a bigger dog than Craig because _ fed their dog much more food.', -1]


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from tqdm.auto import tqdm # Utilisation de tqdm.auto pour une détection automatique dans Colab

# --- Détection GPU/CPU et configuration du périphérique ---
# Cette partie est essentielle pour que PyTorch sache où envoyer les calculs
if torch.cuda.is_available():
    device = torch.device("cuda") # 'cuda' indique le GPU
    print(f"GPU disponible ! Utilisation de : {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu") # 'cpu' indique le CPU si pas de GPU
    print("Pas de GPU disponible, utilisation du CPU.")


tokenizer = AutoTokenizer.from_pretrained("camembert-base")
# >>> MODIFICATION IMPORTANTE ICI : Déplacer le modèle sur le périphérique (GPU si disponible)
model = AutoModel.from_pretrained("camembert-base").to(device)

# === 2. Fonction pour encoder une liste de textes en embeddings BERT par lots ===
def get_bert_embeddings_batch(texts, batch_size=64): # Augmentez le batch_size si vous avez un bon GPU Colab
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Encodage des textes par lots"):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", truncation=True, padding=True)

        # >>> MODIFICATION IMPORTANTE ICI : Déplacer les inputs (tensors) vers le même périphérique que le modèle
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)

        # Ramener le tenseur résultant sur le CPU AVANT de le convertir en NumPy
        batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
        embeddings.append(batch_embeddings)
    return np.vstack(embeddings)

# === 3. Données d'exemple ===
# Assurez-vous que votre 'liste_2D' est bien définie ici ou importée.
# Exemple fictif pour que le code soit exécutable si 'liste_2D' n'est pas définie
# try:
#     _ = liste_2D
# except NameError:
#     # Pour un exemple réaliste si 'liste_2D' n'est pas définie

X_success = [ligne[0] for ligne in liste_2D]
y = [ligne[1] for ligne in liste_2D]




# === 4. Transformer les textes en embeddings ===
# >>> MODIFICATION IMPORTANTE ICI : Vous appeliez get_bert_embeddings_batch sur un seul texte, pas sur la liste
# La ligne originale: X_embed = [get_bert_embeddings_batch(text) for text in tqdm(X_success, desc="Encodage des textes")]
# Ceci est incorrect car get_bert_embeddings_batch attend une LISTE de textes.
# De plus, vous avez une tqdm imbriquée qui n'est pas nécessaire.
# Utilisez directement la fonction de batching sur votre liste X_success :
X_embed_np = get_bert_embeddings_batch(X_success, batch_size=64) # Ajustez batch_size pour votre GPU Colab

# === 5. Utilisation dans scikit-learn ===
X_train, X_test, y_train, y_test = train_test_split(X_embed_np, y, test_size=0.2, random_state=989)

# Scikit-learn (LogisticRegression) utilise le CPU par défaut, et c'est très bien car les données sont déjà sur le CPU (X_embed_np)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


GPU disponible ! Utilisation de : Tesla T4


Encodage des textes par lots:   0%|          | 0/7 [00:00<?, ?it/s]

Accuracy: 0.8235294117647058


In [ ]:

liste1 = ['Rdzqdzqdzqddddddddy brdzqqdzqd d zqd qdzd zdqd qdzdzqdq  dzqd qd ast.']
print(clf.predict(get_bert_embeddings_batch(liste1, batch_size=64)))

Encodage des textes par lots:   0%|          | 0/1 [00:00<?, ?it/s]

[0]


In [ ]:
y_train

['1',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0']

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.80      1.00      0.89        33
           1       0.00      0.00      0.00         8

    accuracy                           0.80        41
   macro avg       0.40      0.50      0.45        41
weighted avg       0.65      0.80      0.72        41



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
y_pred

array(['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0'], dtype='<U1')

In [ ]:
print(y_pred)

['0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0']


In [ ]:
test = ["Albert was better than Paul in college because _ worked harder"]
test = get_bert_embeddings_batch(test)

test = test.reshape(1, -1)
print(test.shape)
test_pred = clf.predict(test)
test_pred

Encodage des textes par lots:   0%|          | 0/1 [00:00<?, ?it/s]

(1, 768)


array(['0'], dtype='<U1')

from logging import raiseExceptions
vrai positifs et négatifs
check : prendre plein des fois la même phrase et voir si c'est bien reconnu
faire un graphe de la probabilité en fonction des différentes itéraiseExceptions


est ce que c'est possible de faire dans l'autre sens ? => partir des embeddings moyens => et arriver à des catégories
à grande dimension, on peut mettre un nb exp de vecteurs orthogonaux
anisotropie => ragarder papier,
on a des connes : car faire des sommes d'objets dans tous les espaces => n'encode rien


faire un classifier pour les femmes et les hommes
voir les corrélations des dimmensions avec le genre
mettre un couche linéaire : 768 dimension => score


réunion vendredi mitn llm : rammener des questions de recherche et des hypothèses
voir le vocabulaire très genré, ex : diet
faire des permutations des prénomask

utiliser she ou he
refaire tourner


In [ ]:
print(X_success)

["Ian volunteered to eat Dennis's menudo after already having a bowl because _ despised eating intestine.", 'He never comes to my home, but I always go to his house because the _ is smaller.', "Kyle doesn't wear leg warmers to bed, while Logan almost always does. _ is more likely to live in a colder climate.", 'The GPS and map helped me navigate home.  I got lost when the _ got turned upside down.', 'Emily looked up and saw Patricia racing by overhead, as _ was on the ramp .', 'The treasury workers took the gold bars off of the trolley and stacked them in the safe until the _ was empty.', 'He thought he had time to take the scenic route to the store, but the _ was under construction.', 'She remembered how annoying it is to dust her wood chair so she bought a plastic table instead.  Cleaning the _ is quick.', 'To overcome behavioural bias, we need to focus more on changing the conscious actions instead of the unconscious actions because the _ actions are voluntary.', 'Megan has a lot le

éviter la comparaison : car c'est trop syntaxique
prendre des sujets plus concrets
bricolage
scolaire
